# Extraccion de medidas de tendencia central: media, mediana y moda, en todas las columnas (.csv's airbnb y ecobici)

## 1. Importar las librerias necesarias

In [40]:
# Import Required Libraries
import pandas as pd
import numpy as np
from scipy import stats
from IPython.display import display

## 2. Cargar los datos

In [41]:
# Cargar datos de airbnb
airbnb = pd.read_csv('listings.csv')

# Cargar datos de ecobici
ecobici = pd.read_csv('2025-06.csv')

## 3. Inspeccionar los datos

In [42]:
# Inspeccionar datos de airbnb
print("Datos de Airbnb:")
print(airbnb.head())
print(airbnb.info())
print(airbnb.describe())

Datos de Airbnb:
      id                         listing_url       scrape_id last_scraped  \
0  35797  https://www.airbnb.com/rooms/35797  20250319150644   2025-03-21   
1  44616  https://www.airbnb.com/rooms/44616  20250319150644   2025-03-20   
2  56074  https://www.airbnb.com/rooms/56074  20250319150644   2025-03-20   
3  67703  https://www.airbnb.com/rooms/67703  20250319150644   2025-03-20   
4  70644  https://www.airbnb.com/rooms/70644  20250319150644   2025-03-22   

            source                                              name  \
0      city scrape                                       Villa Dante   
1  previous scrape                                      Condesa Haus   
2      city scrape              Great space in historical San Rafael   
3  previous scrape                 2 bedroom apt. deco bldg, Condesa   
4      city scrape  Beautiful light Studio Coyoacan- full equipped !   

                                         description  \
0  Dentro de Villa un estudio d

In [43]:
# Inspeccionar datos de ecobici
print("Datos de Ecobici:")
print(ecobici.head())
print(ecobici.info())
print(ecobici.describe())

Datos de Ecobici:
  Genero_Usuario  Edad_Usuario     Bici Ciclo_Estacion_Retiro Fecha_Retiro  \
0              M          30.0  8384919                   552   31/05/2025   
1              F          34.0  4275616                   138   31/05/2025   
2              M          27.0  8370249                   596   31/05/2025   
3              M          27.0  3430139                   304   31/05/2025   
4              M          37.0  8102766                   546   31/05/2025   

  Hora_Retiro Ciclo_EstacionArribo Fecha_Arribo Hora_Arribo  
0    23:52:42                  491   01/06/2025    00:00:06  
1    23:34:45                  036   01/06/2025    00:00:37  
2    23:51:42                  618   01/06/2025    00:01:17  
3    23:46:27                  578   01/06/2025    00:01:17  
4    23:45:14                  052   01/06/2025    00:01:27  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1578804 entries, 0 to 1578803
Data columns (total 9 columns):
 #   Column                 N

## 4. Transformacion de los datos (limpieza) y calculo de media, mediana y moda por columna

A continuacion se aplica una limpieza ligera y luego se calculan las medidas de tendencia central para cada columna numerica y categorica cuando tenga sentido.

In [44]:
# 1) Coercion robusta de tipos (numerico con distintos formatos, % y moneda; booleano; datetime) sin alterar NaN

import re


def _coerce_numeric_locale(s: pd.Series) -> pd.Series:
    s_str = s.astype(str)
    # Quitar espacios (incluyendo no separable), simbolos de moneda comunes y signos de porcentaje
    s_clean = (
        s_str.str.replace(r"[\u00A0\s]", "", regex=True)
             .str.replace(r"[\$€£]", "", regex=True)
             .str.replace(r"[\u20BD\u20B9\u20BA]", "", regex=True)
             .str.replace('%', '', regex=False)
    )
    # Estrategia coma decimal: quitar puntos (miles) y cambiar coma por punto
    s_es = s_clean.str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    n_es = pd.to_numeric(s_es, errors='coerce')
    # Estrategia punto decimal: quitar comas (miles)
    s_en = s_clean.str.replace(',', '', regex=False)
    n_en = pd.to_numeric(s_en, errors='coerce')
    # Elegir mejor parseo
    n = n_es if n_es.notna().sum() > n_en.notna().sum() else n_en
    # Si una proporción significativa tenía % originalmente, dividir por 100
    has_pct = s_str.str.contains('%', regex=False, na=False)
    if has_pct.mean() > 0.2:
        n = n / 100.0
    return n


def coerce_types(df: pd.DataFrame) -> pd.DataFrame:
    from pandas.api.types import is_datetime64_any_dtype
    df = df.copy()
    for col in df.columns:
        s = df[col]
        # Booleanos tipo texto (true/false/si/no/1/0)
        if s.dtype == object:
            s_lower = s.astype(str).str.strip().str.lower()
            true_like = {'true', 'sí', 'si', '1', 'yes', 'y'}
            false_like = {'false', 'no', '0', 'n'}
            mask_known = s_lower.isin(true_like | false_like)
            if mask_known.mean() >= 0.8:
                df[col] = s_lower.isin(true_like).where(mask_known, np.nan)
                continue
        # Numericos con distintos formatos locales, %, moneda
        if s.dtype == object:
            n = _coerce_numeric_locale(s)
            if n.notna().mean() >= 0.3:
                df[col] = n
                continue
        # Datetime (permitir dayfirst para formatos locales)
        if s.dtype == object:
            dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
            if dt.notna().mean() >= 0.3:
                df[col] = dt
                continue
        # Si ya es datetime-like en dtypes antiguos, lo dejamos
        if is_datetime64_any_dtype(s):
            continue
    return df

# Aplicar coercion mejorada sin eliminar columnas/filas ni imputar
airbnb = coerce_types(airbnb)
ecobici = coerce_types(ecobici)

# NOTA: Conservamos NaN para reflejarlos en el resumen; no se imputan ni se eliminan filas/columnas aquí.

/tmp/ipykernel_13169/3851338286.py:52: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
/tmp/ipykernel_13169/3851338286.py:52: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt = pd.to_datetime(s, errors='coerce', dayfirst=True, infer_datetime_format=True)
/tmp/ipykernel_13169/3851338286.py:52: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  d

In [45]:
# 5. Calculo de media, mediana y moda para todas las columnas (soporte para numericos, booleanos y fechas)

from collections import OrderedDict
from pandas.api.types import is_numeric_dtype, is_bool_dtype, is_datetime64_any_dtype


def summarize_central_tendency(df: pd.DataFrame, name: str) -> pd.DataFrame:
    """
    Devuelve un DataFrame con indice=columnas y columnas:
      [tipo, media, mediana, moda, conteo_moda, n_no_nulos, cobertura, motivo_no_aplica]
    Reglas:
      - Numericas y booleanas: media/mediana numericas; moda y su frecuencia.
      - Datetime: media y mediana como timestamps; moda y su frecuencia.
      - Otras: media/mediana = NaN; se calcula moda si existe.
    """
    rows = []
    for col in df.columns:
        series = df[col]
        s_non_null = series.dropna()
        dtype_str = str(series.dtype)
        coverage = series.notna().mean()
        motivo = ''
        mean_val = np.nan
        median_val = np.nan
        mode_val = np.nan
        mode_count = 0

        if len(s_non_null) == 0:
            motivo = 'sin datos'
        elif is_datetime64_any_dtype(series):
            # pandas soporta mean/median para datetime
            try:
                mean_val = series.mean()
            except Exception:
                mean_val = np.nan
            try:
                median_val = series.median()
            except Exception:
                median_val = np.nan
            try:
                mv = series.mode(dropna=True)
                if len(mv) > 0:
                    mode_val = mv.iloc[0]
                    mode_count = (series == mode_val).sum()
            except Exception:
                pass
        elif is_bool_dtype(series) or is_numeric_dtype(series):
            try:
                mean_val = series.mean()
            except Exception:
                mean_val = np.nan
            try:
                median_val = series.median()
            except Exception:
                median_val = np.nan
            try:
                mv = series.mode(dropna=True)
                if len(mv) > 0:
                    mode_val = mv.iloc[0]
                    mode_count = (series == mode_val).sum()
            except Exception:
                pass
        else:
            motivo = 'no numerica ni fecha'
            try:
                mv = series.mode(dropna=True)
                if len(mv) > 0:
                    mode_val = mv.iloc[0]
                    mode_count = (series == mode_val).sum()
            except Exception:
                pass

        rows.append(OrderedDict(
            columna=col,
            tipo=dtype_str,
            media=mean_val,
            mediana=median_val,
            moda=mode_val,
            conteo_moda=mode_count,
            n_no_nulos=series.notna().sum(),
            cobertura=coverage,
            motivo_no_aplica=motivo
        ))

    out = pd.DataFrame(rows).set_index('columna')
    out.index.name = f"columnas_{name}"
    return out

res_airbnb = summarize_central_tendency(airbnb, 'airbnb')
res_ecobici = summarize_central_tendency(ecobici, 'ecobici')

# Mostrar tablas formateadas: NaN visibles y cobertura con formato porcentaje
fmt = {
    'media': '{:,.4f}'.format,
    'mediana': '{:,.4f}'.format,
    'conteo_moda': '{:,.0f}'.format,
    'n_no_nulos': '{:,.0f}'.format,
    'cobertura': '{:.1%}'.format,
}

print("Resumen Airbnb (primeras 5 columnas):")
display(
    res_airbnb.head(5)
        .style
        .format(fmt, na_rep='NaN')
        .apply(lambda s: np.where(s.isna(), '', ''), axis=0)
)

print("Resumen Ecobici (primeras 30 filas):")
display(
    res_ecobici.head(5)
        .style
        .format(fmt, na_rep='NaN')
        .apply(lambda s: np.where(s.isna(), '', ''), axis=0)
)

Resumen Airbnb (primeras 5 columnas):


,tipo,media,mediana,moda,conteo_moda,n_no_nulos,cobertura,motivo_no_aplica
columnas_airbnb,,,,,,,,
id,int64,"604,134,850,682,801,536.0000","739,843,990,759,705,856.0000",35797,1,"26,067",100.0%,
listing_url,object,NaN,NaN,https://www.airbnb.com/rooms/1000010393808461294,1,"26,067",100.0%,no numerica ni fecha
scrape_id,int64,"20,250,319,150,644.0000","20,250,319,150,644.0000",20250319150644,"26,067","26,067",100.0%,
last_scraped,datetime64[ns],",.4f",",.4f",2025-03-20 00:00:00,"12,569","26,067",100.0%,
source,object,NaN,NaN,city scrape,"22,068","26,067",100.0%,no numerica ni fecha


Resumen Ecobici (primeras 30 filas):


,tipo,media,mediana,moda,conteo_moda,n_no_nulos,cobertura,motivo_no_aplica
columnas_ecobici,,,,,,,,
Genero_Usuario,object,NaN,NaN,M,"1,075,077","1,578,802",100.0%,no numerica ni fecha
Edad_Usuario,float64,34.0537,32.0000,30.000000,"80,698","1,578,704",100.0%,
Bici,int64,"5,470,371.3543","5,468,426.0000",5014865,386,"1,578,804",100.0%,
Ciclo_Estacion_Retiro,float64,291.9378,252.0000,548.000000,"9,245","1,519,018",96.2%,
Fecha_Retiro,datetime64[ns],",.4f",",.4f",2025-06-04 00:00:00,"73,465","1,578,804",100.0%,


## 5. Guardado de las metricas  en csv para mejor visualizacion

In [46]:
# Exportar resultados a CSV
res_airbnb.to_csv('resumen_central_tendencia_airbnb.csv')
res_ecobici.to_csv('resumen_central_tendencia_ecobici.csv')

print("Archivos exportados: resumen_central_tendencia_airbnb.csv y resumen_central_tendencia_ecobici.csv")

Archivos exportados: resumen_central_tendencia_airbnb.csv y resumen_central_tendencia_ecobici.csv
